#### Import dependencies:

In [2]:
import sys #access to system parameters https://docs.python.org/3/library/sys.html
print("Python version: {}". format(sys.version))

import pandas as pd #collection of functions for data processing and analysis modeled after R dataframes with SQL like features
print("pandas version: {}". format(pd.__version__))

import matplotlib #collection of functions for scientific and publication-ready visualization
import matplotlib as plt
print("matplotlib version: {}". format(matplotlib.__version__))

import numpy as np #foundational package for scientific computing
print("NumPy version: {}". format(np.__version__))

import scipy as sp #collection of functions for scientific computing and advance mathematics
print("SciPy version: {}". format(sp.__version__)) 

import IPython
from IPython import display #pretty printing of dataframes in Jupyter notebook
print("IPython version: {}". format(IPython.__version__)) 

import sklearn #collection of machine learning algorithms
print("scikit-learn version: {}". format(sklearn.__version__))

#misc libraries
import random
import time
#ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('-'*25)
import re
import os
import matplotlib.pyplot as plt
%matplotlib inline

Python version: 3.8.5 (default, Sep  4 2020, 02:22:02) 
[Clang 10.0.0 ]
pandas version: 1.1.3
matplotlib version: 3.3.2
NumPy version: 1.19.5
SciPy version: 1.5.2
IPython version: 7.19.0
scikit-learn version: 0.23.2
-------------------------


#### Download data file if it has not been downloaded already:

In [3]:
#url = 'https://data.cityofnewyork.us/api/views/qgea-i56i/rows.csv?accessType=DOWNLOAD'
fn_src = 'https://data.cityofnewyork.us/api/views/57p3-pdcj/rows.csv?accessType=DOWNLOAD'
fn_dst = 'NYPD-B-Summons-Year-to-Date.csv'

from six.moves import urllib

if os.path.isfile(fn_dst):
    print('File %s has already been downloaded' % fn_dst)
else:
    print('Fetching file %s[2.4GB]. This may take a while...' % fn_dst)
    urllib.request.urlretrieve(fn_src, fn_dst)
    print('File %s has been downloaded' % fn_dst)

Fetching file NYPD-B-Summons-Year-to-Date.csv[2.4GB]. This may take a while...
File NYPD-B-Summons-Year-to-Date.csv has been downloaded


#### Open a stream to the data file so we don't have to load the whole data set into main memory

In [4]:
from openclean.pipeline import stream
ds_full = stream(fn_dst)

In [5]:
print(f'The dataset contains {ds_full.count():,} rows.')

The dataset contains 377,501 rows.


In [6]:
ds_full.head()

,EVNT_KEY,VIOLATION_DATE,VIOLATION_TIME,CHG_LAW_CD,VIOLATION_CODE,VEH_CATEGORY,CITY_NM,RPT_OWNING_CMD,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Location Point,JURIS_CD
0,234185278,09/28/2021,19:02:56,VTL,1180D,CAR/SUV,BROOKLYN,412,,,,,,1
1,234116090,09/27/2021,13:06:23,VTL,4011A,CAR/SUV,BRONX,47,,,,,,1
2,234115701,09/27/2021,08:27:18,VTL,1180D,CAR/SUV,BRONX,411,,,,,,1
3,234066027,09/26/2021,02:21:24,VTL,3752A1,CAR/SUV,BROOKLYN,412,,,,,,1
4,234074374,09/26/2021,08:37:58,VTL,1180D,CAR/SUV,BROOKLYN,412,,,,,,1
5,234065505,09/25/2021,23:50:00,VTL,3752A1,CAR/SUV,QUEENS,101,,,,,,1
6,234086888,09/25/2021,22:23:00,VTL,1110A,CAR/SUV,BROOKLYN,73,,,,,,1
7,233975375,09/23/2021,17:20:00,VTL,1229C1C,CAR/SUV,BRONX,50,,,,,,1
8,233931258,09/22/2021,21:14:15,VTL,4011A,CAR/SUV,QUEENS,108,,,,,,1
9,233869480,09/21/2021,19:18:00,VTL,1180D,CAR/SUV,BROOKLYN,412,,,,,,1


#### Use openclean to profile the data set:

In [7]:
from openclean.profiling.column import DefaultColumnProfiler

#profiles = ds.profile(default_profiler=DefaultColumnProfiler)
profiles = ds_full.profile(default_profiler=DefaultColumnProfiler)

In [8]:
profiles.stats()

,total,empty,distinct,uniqueness,entropy
EVNT_KEY,377501,0,377501,1.000000,18.526121
VIOLATION_DATE,377501,0,273,0.000723,7.972660
VIOLATION_TIME,377501,0,58833,0.155849,13.781449
CHG_LAW_CD,377501,0,2,0.000005,0.559410
VIOLATION_CODE,377501,0,375,0.000993,5.079496
VEH_CATEGORY,377501,0,6,0.000016,0.772714
CITY_NM,377501,8,6,0.000016,2.298109
RPT_OWNING_CMD,377501,0,83,0.000220,6.186397
X_COORD_CD,377501,1966,17489,0.046571,11.514411
Y_COORD_CD,377501,1966,18113,0.048233,11.536104


In [9]:
# Print the most frequent data type for each column.

print('Schema\n------')
for col in ds_full.columns:
    p = profiles.column(col)
    print("  '{}' ({})".format(col, p['datatypes']['distinct'].most_common(1)[0][0]))

Schema
------
  'EVNT_KEY' (int)
  'VIOLATION_DATE' (date)
  'VIOLATION_TIME' (date)
  'CHG_LAW_CD' (str)
  'VIOLATION_CODE' (str)
  'VEH_CATEGORY' (str)
  'CITY_NM' (str)
  'RPT_OWNING_CMD' (int)
  'X_COORD_CD' (int)
  'Y_COORD_CD' (int)
  'Latitude' (float)
  'Longitude' (float)
  'Location Point' (str)
  'JURIS_CD' (int)


### Field Name & Description:
    'CMPLNT_NUM' (int)                 Complaint Number
    'CMPLNT_FR_DT' (date)              Complaint From Date
    'CMPLNT_FR_TM' (date)              Complaint From Time
    'CMPLNT_TO_DT' (date)              Complaint To Date
    'CMPLNT_TO_TM' (date)              Complaint To Time
    'ADDR_PCT_CD' (int)                Code of Precinct in which the Incident Occured
    'RPT_DT' (date)                    Report Date
    'KY_CD' (int)                      "Key Code": Offense Classification Code (3 digits)
    'OFNS_DESC' (str)                  Offense Description
    'PD_CD' (int)                      PD Code of Offense. More granular than Key Code
    'PD_DESC' (str)                    PD Description of Offense.
    'CRM_ATPT_CPTD_CD' (str)           Whether Crime was Atempted or Completed (values: 'COMPLETED', 'ATTEMPTED')
    'LAW_CAT_CD' (str)                 Level of Offense (values: 'FELONY', 'VIOLATION', 'MISDEMEANOR')
    'BORO_NM' (str)                    Name of Borough in which Incident Occurred
    'LOC_OF_OCCUR_DESC' (str)          Description of where the incident occurred with respect to the premises
                                       (values:'FRONT OF', 'REAR OF', 'OUTSIDE', 'INSIDE', 'OPPOSITE OF')
    'PREM_TYP_DESC' (str)              Description of the type of premises in which the Incident Occurred
    'JURIS_DESC' (str)                 Description of Jurisdiction in which Incident Occurred
    'JURISDICTION_CODE' (int)          Jurisdiction Code
    'PARKS_NM' (str)                   Name of Park in which Incident Occurred, if Applicable
    'HADEVELOPT' (str)                 Name of NYCHA Housing Development in which Incident Occurred, if Applicable
    'HOUSING_PSA' (int)                Housing PSA
    'X_COORD_CD' (int)                 X-coordinate, New York State Plane Coordinate System
    'Y_COORD_CD' (int)                 Y-coordinate, New York State Plane Coordinate System
    'SUSP_AGE_GROUP' (int)             Age Group of Suspect
    'SUSP_RACE' (str)                  Race of Suspect
    'SUSP_SEX' (str)                   Sex of Suspect
    'TRANSIT_DISTRICT' (int)           Transit-District code
    'Latitude' (float)                 Global Latitude of Location where Incident Occurred
    'Longitude' (float)                Global Longitude of Location where Incident Occured
    'Lat_Lon' (str)                    'Latitude' and 'Longitude' together
    'PATROL_BORO' (str)                Patrol Borough
    'STATION_NAME' (str)               Station Name
    'VIC_AGE_GROUP' (int)              Age Group of Victim
    'VIC_RACE' (str)                   Race of Victim
    'VIC_SEX' (str)                    Sex of Victim
    
    
    (Note: some field descriptions were taken from https://www1.nyc.gov/assets/nypd/downloads/pdf/analysis_and_planning/incident_level_data_footnotes.pdf)
   

## Profiling & Cleaning of each field in the data set

In [10]:
profiles.minmax('EVNT_KEY')

,min,max
int,144077558,234617922


In [12]:
profiles.minmax('VIOLATION_DATE')

,min,max
date,2021-01-01,2021-09-30


In [13]:
from datetime import datetime
#datetime.strptime('2014-12-04', '%Y-%m-%d').date()


def validate_date(sourceValue):
    dummy_date = datetime.strptime("01/01/1000", '%m/%d/%Y').date()
    
    try:
        source_date = sourceValue
        start_date = datetime.strptime("01/01/2021", '%m/%d/%Y').date()
        end_date = datetime.strptime("09/30/2019", '%m/%d/%Y').date()
    
        if source_date == '':
            source_date = dummy_date
        
        source_date = datetime.strptime(sourceValue, '%m/%d/%Y').date()
        
        if source_date >= start_date and source_date <= end_date:
            return source_date
        else:
            return dummy_date
    except:
        return dummy_date
    

ds_full = ds_full.update('VIOLATION_DATE', validate_date)

In [14]:
profiles.minmax('VIOLATION_TIME')

,min,max
date,2021-12-11,2021-12-11 23:59:59


In [15]:
profiles.minmax('RPT_OWNING_CMD')

,min,max
int,1,442


In [16]:
profiles.column('RPT_OWNING_CMD').get('topValues')

[('411', 14787),
 ('413', 14328),
 ('412', 14086),
 ('442', 10940),
 ('70', 7123),
 ('40', 7060),
 ('14', 6782),
 ('75', 6752),
 ('17', 6447),
 ('113', 6148)]

In [19]:
profiles.minmax('CHG_LAW_CD')

,min,max
str,NYC,VTL


In [20]:
profiles.column('CHG_LAW_CD').get('topValues')

[('VTL', 328154), ('NYC', 49347)]

In [21]:
profiles.column('VEH_CATEGORY').get('topValues')

[('CAR/SUV', 328738),
 ('MCL', 20861),
 ('TRUCK/BUS', 15411),
 ('BIKE', 11202),
 ('ZZZ', 1209),
 ('ATV', 80)]

In [22]:
ds_full.distinct('VEH_CATEGORY')

Counter({'CAR/SUV': 328738,
         'MCL': 20861,
         'TRUCK/BUS': 15411,
         'BIKE': 11202,
         'ZZZ': 1209,
         'ATV': 80})

In [23]:
profiles.minmax('VIOLATION_CODE')

,min,max
str,10127B,6001A
int,413,375415
float,4.08e+07,4.08e+07


In [24]:
profiles.column('VIOLATION_CODE').get('topValues')

[('1180D', 64518),
 ('1110A', 41947),
 ('5091', 24723),
 ('1225D', 23730),
 ('403A1', 18785),
 ('1111D1N', 18638),
 ('1229C3A', 18078),
 ('406A1', 13293),
 ('1225C2A', 11155),
 ('1163AT', 9512)]

In [27]:
ds_full.distinct('CITY_NM')

Counter({'BROOKLYN': 120032,
         'BRONX': 61634,
         'QUEENS': 88150,
         'MANHATTAN': 71581,
         'NEW YORK': 8872,
         'STATEN ISLAND': 27232})

In [26]:
def validate_boro_nm(sourceValue):
    UNKNOWN = "NEW YORK"
    
    try:
        values = ["BRONX","BROOKLYN","MANHATTAN","QUEENS","STATEN ISLAND","NEW YORK"]
        
        if sourceValue in values:
            return sourceValue
        else:
            return UNKNOWN
    except:
        return UNKNOWN

    
ds_full = ds_full.update('CITY_NM', validate_boro_nm)

In [28]:
profiles.column('JURIS_CD').get('topValues')

[('1', 377501)]

In [30]:
ds_full.distinct('JURIS_CD')

Counter({'1': 377501})

In [31]:
profiles.minmax('X_COORD_CD')

,min,max
int,914210,1067113


In [32]:
profiles.minmax('Y_COORD_CD')

,min,max
int,122552,271128


In [33]:
profiles.minmax('Latitude')

,min,max
float,40.502784,40.910819


In [34]:
def validate_lat(sourceValue):
    UNKNOWN = -999
    
    try:
        if sourceValue >= 40 and sourceValue <= 41:  
            return sourceValue
        else:
            return UNKNOWN
    except:
        return UNKNOWN

    
ds_full = ds_full.update('Latitude', validate_lat)

In [35]:
profiles.minmax('Longitude')

,min,max
float,-74.251853,-73.700985


In [36]:
def validate_long(sourceValue):
    UNKNOWN = -999
    
    try:
        if sourceValue >= -74 and sourceValue <= -73:  
            return sourceValue
        else:
            return UNKNOWN
    except:
        return UNKNOWN

    
ds_full = ds_full.update('Latitude', validate_long)

In [37]:
profiles.minmax('Location Point')

,min,max
str,POINT (-73.70098487999996 40.738904895000076),POINT (-74.25185282599993 40.502813784000075)


In [38]:
SELECTED_COLUMNS = [\
    'EVNT_KEY',\
    'VIOLATION_DATE',\
    'VIOLATION_TIME',\
    'CHG_LAW_CD',\
    'VIOLATION_CODE',\
    'VEH_CATEGORY',\
    'CITY_NM',\
    'RPT_OWNING_CMD',\
    'X_COORD_CD',\
    'Y_COORD_CD',\
    'Latitude',\
    'Longitude',\
    'Location Point',\
    'JURIS_CD',\

]

ds_full = ds_full.select(SELECTED_COLUMNS)

In [39]:
ds_full

# VALIDATING DATA AFTER CLEANING

In [40]:
data_df = pd.read_csv("NYPD-B-Summons-Year-to-Date.csv")

In [42]:
pd.to_numeric(data_df["JURIS_CD"])

0         1
1         1
2         1
3         1
4         1
         ..
377496    1
377497    1
377498    1
377499    1
377500    1
Name: JURIS_CD, Length: 377501, dtype: int64

In [44]:
data_df = data_df.dropna()

In [45]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 375527 entries, 32 to 377500
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   EVNT_KEY        375527 non-null  int64  
 1   VIOLATION_DATE  375527 non-null  object 
 2   VIOLATION_TIME  375527 non-null  object 
 3   CHG_LAW_CD      375527 non-null  object 
 4   VIOLATION_CODE  375527 non-null  object 
 5   VEH_CATEGORY    375527 non-null  object 
 6   CITY_NM         375527 non-null  object 
 7   RPT_OWNING_CMD  375527 non-null  int64  
 8   X_COORD_CD      375527 non-null  float64
 9   Y_COORD_CD      375527 non-null  float64
 10  Latitude        375527 non-null  float64
 11  Longitude       375527 non-null  float64
 12  Location Point  375527 non-null  object 
 13  JURIS_CD        375527 non-null  int64  
dtypes: float64(4), int64(3), object(7)
memory usage: 43.0+ MB


In [46]:
ds_full_jc = stream(data_df)
profiles = ds_full_jc.profile(default_profiler=DefaultColumnProfiler)
profiles.stats()

,total,empty,distinct,uniqueness,entropy
EVNT_KEY,375527,0,375527,1.000000,18.518557
VIOLATION_DATE,375527,0,273,0.000727,7.973223
VIOLATION_TIME,375527,0,58758,0.156468,13.781430
CHG_LAW_CD,375527,0,2,0.000005,0.559163
VIOLATION_CODE,375527,0,375,0.000999,5.078146
VEH_CATEGORY,375527,0,6,0.000016,0.771661
CITY_NM,375527,0,6,0.000016,2.298166
RPT_OWNING_CMD,375527,0,83,0.000221,6.186042
X_COORD_CD,375527,0,17489,0.046572,11.514424
Y_COORD_CD,375527,0,18113,0.048234,11.536118


In [47]:
data_df.to_csv('filtered5.csv')